### INST326 OOP Project 03

Rename this notebook, replacing "_Assignment" with "_YourName"<br>
Insert Signature Block Here

#### Myles Sartor
> INST326 Section 0104   
> Myles Sartor, Dany Drammeh, Inmar Lizama
<br/>Project 03
<br/>11/21/24 
#### Honor Pledge
> I pledge that the work contained in this assignment is my own, and that I have complied with University and course policies on academic integrity, and AI use.

### The Project
Everyone will do the same project this time. This is a group project, so you must work in your assigned groups. Include the link to your group's GitHub repository (one link per group). Use comments in your code to document your solution. If you need to write comments to the grader, add a markdown cell immediately above your code solution and add your comments there. Be sure to read and follow all the requirements and the Notebook Instructions at the bottom of this notebook. Your grade may depend on it!

#### 1. A Scheduling Program>  My wife is responsible for scheduling caregivers for her 93 year-old mother. Currently she writes out the schedule on a monthly calendar and photocopies it for everyone. I want all of you to help me write a program to help her with scheduling. While this is a specific application, this program will be broadly useful and adaptable to any scheduling needs for small businesses, clubs, and more.

#### Requirements
>  Care is required 12 hours per day, 7 days a week. There are two shifts each day: 7:00 AM - 1:00 PM, and 1:00 PM to 7:00 PM. There are a total of 8 caregivers. Some are family members and some are paid. Each caregiver has their own availability for shifts that is generally the same from month to month, but there are exceptions for work, vacations, and other responsibilities. Your program should do the following:
> 1. Manage caregivers and their schedules. Attributes include: name, phone, email, pay rate, and hours.
> 2. Each caregiver should have their own availability schedule where they can indicate their availability for each shift. Availability categories are 'preferred', 'available' (default), and 'unavailable'.
> 3. Create a care schedule that covers AM and PM shifts and displays caregiver names on a calendar (see example). The schedule should accomodate caregivers' individual schedules and availability preferences. The python calendar module provides options for creating HTML calendars. Sample code for the HTML calendar is in the project folder.
> 4. Paid caregivers are paid weekly at $20/hr. Your program should calculate weekly pay based on assigned hours. Provide a separate pay report that lists weekly (gross: hours x rate) amounts to each caregiver, along with weekly and monthly totals. The report can be a text document, or presented in GUI or HTML format. 

In [127]:
import calendar
from tkinter import *
from tkinter import filedialog



class Person: # person class for each worker
    def __init__(self, name, phone, email):
        self.name = name
        self.phone = phone
        self.email = email


class Caregiver(Person): # cargiver class inherits person class attributes
    def __init__(self, name, phone, email, pay_rate = 20.0, hours = 0):
        super().__init__(name, phone, email)
        self.pay_rate = float(pay_rate)
        self.availability = [["Available"] * 2 for _ in range(7)]  # 7 days, 2 shifts per day
        self.hours = float(hours)  # hours for payroll

    def update_details(self, name, phone, email, pay_rate): # method for updating details of caregiver
        self.name = name
        self.phone = phone
        self.email = email
        self.pay_rate = float(pay_rate)

    def set_availability(self, availability): # method to set caregiver availability 
        self.availability = availability

    def calculate_weekly_pay(self): # method to determine pay based on hours worked
        return self.hours * self.pay_rate


class CaregiverManager: # class to deal with GUI for caregiver manager application 
    def __init__(self):
        # main window 
        self.master = Tk()
        self.master.title("Caregiver Manager")
        self.master.geometry("900x900")

        # Storing caregivers in a list
        self.caregivers = []

        # GUI labels for name, phone number, email, and pay rate
        self.create_widgets()

    def create_widgets(self): # creating labels, buttons, entry, and string variables to store and change later
        Label(self.master, text = "Name").grid(row = 0, column = 0, padx = 10, pady = 5)
        Label(self.master, text = "Phone").grid(row = 1, column = 0, padx = 10, pady = 5)
        Label(self.master, text = "Email").grid(row = 2, column = 0, padx = 10, pady = 5)
        Label(self.master, text = "Pay Rate").grid(row = 3, column = 0, padx = 10, pady = 5)
        Label(self.master, text = "Weekly Hours").grid(row = 4, column = 0, padx = 10, pady = 5)

        
        self.name_var = StringVar() # initializing string varaibles that hold and link to entry, button, and label widgets (changes happen in program and GUI)
        self.phone_var = StringVar()
        self.email_var = StringVar()
        self.pay_rate_var = StringVar(value = "20.0") # holds default pay rate in float to account for cents
        self.hours_var = StringVar(value = "0")

        # entry input widgets 
        Entry(self.master, text = self.name_var).grid(row = 0, column = 1, padx = 10, pady = 5)
        Entry(self.master, text = self.phone_var).grid(row = 1, column = 1, padx = 10, pady = 5)
        Entry(self.master, text = self.email_var).grid(row = 2, column = 1, padx = 10, pady = 5)
        Entry(self.master, text = self.pay_rate_var).grid(row = 3, column = 1, padx = 10, pady = 5)
        Entry(self.master, text = self.hours_var).grid(row = 4, column = 1, padx = 10, pady = 5)


       

        # selectable listbox widget for caregivers
        self.caregiver_list = Listbox(self.master, width = 50, height = 15)
        self.caregiver_list.grid(row = 0, column = 2, rowspan = 5, padx = 10, pady = 5)
        self.caregiver_list.bind("<<ListboxSelect>>", self.add_attributes)

        # buttons widgets, with commands from methods below once pressed
        Button(self.master, text = "Add Caregiver", command = self.add_caregiver).grid(row = 11, column = 0, columnspan = 2, pady = 10)
        Button(self.master, text = "Update Caregiver", command = self.update_caregiver).grid(row = 5, column = 0, columnspan = 2, pady = 5)
        Button(self.master, text = "Delete Caregiver", command = self.delete_caregiver).grid(row = 6, column = 0, columnspan = 2, pady = 5)
        Button(self.master, text = "Set Availability", command = self.availability_window).grid(row = 7, column = 0, columnspan = 2, pady = 5)
        Button(self.master, text = "Generate Schedule", command = self.generate_schedule).grid(row = 8, column = 0, columnspan = 2, pady = 5)
        Button(self.master, text = "Pay Report", command = self.payroll_report).grid(row = 9, column = 0, columnspan = 2, pady = 5)
        Button(self.master, text = "Update Hours", command = self.update_hours).grid(row = 10, column = 0, columnspan = 2, pady = 5)


        self.load_caregivers() # loading current caregivers into the selectable listbox widget

    def add_caregiver(self):
        name = self.name_var.get()
        phone = self.phone_var.get()
        email = self.email_var.get()
        pay_rate = float(self.pay_rate_var.get())
        hours = float(self.hours_var.get())

        if not name or not phone or not email:
            print("Missing input")
            return
            

        caregiver = Caregiver(name, phone, email, pay_rate, hours)
        self.caregivers.append(caregiver)
        self.remove()
        self.load_caregivers()

    def update_hours(self): # separate handling of caregiver's hours
        if not self.caregiver_list.curselection(): # curselection selects items in listbox 
            print("Select a caregiver")
            return

        list_index = self.caregiver_list.curselection()[0]
        caregiver = self.caregivers[list_index]

        try: # make sure a number is entered in this field with try excpet block 
            caregiver.hours = float(self.hours_var.get())  # update caregiver's hours
            print(f"Updated {caregiver.name}'s hours to {caregiver.hours}")
        except ValueError:
            print("Enter a number.")


    def add_attributes(self, attributes): # adds attributes into input fields when caregiver is selected so that things can be updated later
        if not self.caregiver_list.curselection():
            return

        list_index = self.caregiver_list.curselection()[0]
        caregiver = self.caregivers[list_index]
# setting entry fields to the original values for the caregiver selected (shows everything you entered before)
        self.name_var.set(caregiver.name)
        self.phone_var.set(caregiver.phone)
        self.email_var.set(caregiver.email)
        self.pay_rate_var.set(caregiver.pay_rate)
        self.hours_var.set(caregiver.hours)

    def update_caregiver(self): # handles the updating of caregiver information 
        if not self.caregiver_list.curselection():
            print("Select a caregiver")
            return

        list_index = self.caregiver_list.curselection()[0]
        caregiver = self.caregivers[list_index]

        caregiver.update_details(self.name_var.get(), self.phone_var.get(), self.email_var.get(), self.pay_rate_var.get())

        self.remove()
        self.load_caregivers()

    def delete_caregiver(self): # delete cargiver selected 
        if not self.caregiver_list.curselection():
            print("Select a caregiver")
            return

        list_index = self.caregiver_list.curselection()[0]
        self.caregivers.pop(list_index)
# removing entry details for caregiver and then loading the set of caregivers again 
        self.remove()
        self.load_caregivers()

    def load_caregivers(self): # delete and repopulate caregiver listbox to properly load the most current updated list of caregivers
        self.caregiver_list.delete(0, END) # beginning to end
        for caregiver in self.caregivers:
            self.caregiver_list.insert(END, caregiver.name)

    def remove(self): # remove entry details after adding or updating caregiver to allow for new entries
        self.name_var.set("")
        self.phone_var.set("")
        self.email_var.set("")
        self.pay_rate_var.set("20.0")

    def availability_window(self): # opens window showing availability of caregivers
        if not self.caregiver_list.curselection():
            print("Select a caregiver")
            return

        list_index = self.caregiver_list.curselection()[0]
        caregiver = self.caregivers[list_index]

        availability_window = Toplevel(self.master) # child window, inherits functionality of main window 
        availability_window.title(f"Set Availability for {caregiver.name}")
        availability_window.geometry("400x400")

        shifts = ["AM", "PM"]
        days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        availability_options = ["Preferred", "Available", "Unavailable"]
# could potentially add separate class for availability
        def set_availability(): # save availability of caregiver for the week, functionality nested inside main availability window
            availability = [
                [var.get() for var in day_vars] for day_vars in day_availability_vars
            ]
            caregiver.set_availability(availability)
            availability_window.destroy()

        day_availability_vars = [] # empty list for day availability 
        for i, day in enumerate(days): # cycles through days and gives a label to all of them for availability to be shown with drop down menus
            Label(availability_window, text = day).grid(row = i, column = 0, padx = 10, pady = 5)
            day_vars = [] # empty list for days, nested inside to account for menu 
            for j, shift in enumerate(shifts):
                var = StringVar(value = caregiver.availability[i][j])
                OptionMenu(availability_window, var, *availability_options).grid(row = i, column = j + 1, padx = 5, pady = 5) # option menu widget 
                day_vars.append(var)
            day_availability_vars.append(day_vars)
            
        # button to save availability options set in menu widget 
        Button(availability_window, text = "Save", command = set_availability).grid(row = len(days), column = 0, columnspan = 3, pady = 10)

    def generate_schedule(self): # method to make schedule for each day in HTML file, with given hours initially set to zero for each unless specified 
        for caregiver in self.caregivers:
            caregiver.hours = 0

        
        schedule = [["", ""] for _ in range(7)]
        shifts = ["AM", "PM"]

        for day in range(7):
            for shift in range(2):
                preferred = [caregiver for caregiver in self.caregivers if caregiver.availability[day][shift] == "Preferred"]
                available = [caregiver for caregiver in self.caregivers if caregiver.availability[day][shift] == "Available"]

                if preferred:
                    selected = preferred[0]
                elif available:
                    selected = available[0]
                else:
                    selected = None

                if selected:
                    schedule[day][shift] = selected.name
                    selected.hours += 6  # Allocate 6 hours for each shift

        self.create_html_calendar(schedule)
        
# creating html calendar for each day of the week and shift 
    def create_html_calendar(self, schedule):
        days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        shifts = ["AM", "PM"]
        calendar_html = "<html><head><title>Care Schedule</title></head><body>"
        calendar_html += "<h1>Weekly Care Schedule</h1><table border='1'><tr><th>Day</th><th>AM Shift</th><th>PM Shift</th></tr>"

        for day, (am, pm) in enumerate(schedule):
            calendar_html += f"<tr><td>{days[day]}</td><td>{am}</td><td>{pm}</"


        calendar_html += "</table></body></html>"

        # writing out caregiver information and schedule into html file 
        file_path = filedialog.asksaveasfilename(defaultextension = ".html", filetypes = [("HTML files", "*.html")])
        if file_path:
            with open(file_path, "w") as file: # writing into this new html file
                file.write(calendar_html)
            print(f"Schedule in {file_path}")

    def payroll_report(self): # function for new payroll window to open as a scrollable 
        payroll_report_window = Tk() # separate window 
        payroll_report_window.title("Payroll Report")
        payroll_report_window.geometry("600x600")
    
        # scroll bar and text widget for payroll report
        text_area = Text(payroll_report_window, wrap = "word") # go to next line at the edge of window
        scroll_bar = Scrollbar(payroll_report_window, orient = VERTICAL, command = text_area.yview)
        text_area.configure(yscrollcommand = scroll_bar.set) # configure the text area to work with the scroll bar
        scroll_bar.pack(side = RIGHT, fill = Y) # packing scrollbar to the right side, filling entire Y space
        text_area.pack(expand = True, fill = "both") # packing text area to the entire space, fills both X and Y planes
    # setting pay to zero for manipulation later
        total_weekly_pay = 0.0
        total_monthly_pay = 0.0
        report = [] # list to hold pay report information
    
        for caregiver in self.caregivers:
            weekly_pay = caregiver.hours * caregiver.pay_rate
            monthly_pay = weekly_pay * 4
            total_weekly_pay += weekly_pay
            total_monthly_pay += monthly_pay
    # adding all the name, pay and hours to report 
            report.append(
                f"Name: {caregiver.name}\n"
                f"Weekly Hours: {caregiver.hours} hours\n"
                f"Weekly Pay: ${weekly_pay:.2f}\n" # 2 decimal places rounding
                f"Monthly Pay: ${monthly_pay:.2f}\n\n"
            )
    # adding totals for weekly and monthly pay to the report
        report.append(f"Total Weekly Pay: ${total_weekly_pay:.2f}\n")
        report.append(f"Total Monthly Pay: ${total_monthly_pay:.2f}\n")
    
        text_area.insert("1.0", "\n".join(report)) # inserting text from the report list at the start, with new line separation
        text_area.configure(state = "disabled") # read only text area


    def run(self): # run method for program
        self.master.mainloop()


# execute caregiver manager
if __name__ == "__main__":
    app = CaregiverManager()
    app.run()



#### Group Requirements
>  1. Your submitted project should follow OOP principles like abstraction, encapsulation, inheritance, and polymorphism as appropriate. Your program should use classes. 
>  2. Select a group leader who will host the group's project repository on their GitHub.
>  3. Create the group repository and add a main program document. See example.
>  4. Create branches off the main program for each group member, and assign part of the program to each member.
>  5. Each member should work on their branch.
>  6. When each member is finished, merge the branches back into the main program. You may use 'merge' or 'pull requests', your choice.
>  7. iterate and debug as necessary.

#### Working with HTML
> Since this is a course on python, not HTML, you are not expected to know HTML. Therefore, you may copy applicable portions of the sample code or use AI to write the HTML portions of your application. Ypu should write the main python code yourself.


#### What you need to turn in
>  This is a group project. There will be one submission per group. Your submission will be graded as a group.
>  1. Include your group number and the names of all group members in the signature block at the top of this notebook.
>  2. In the cell below, paste the link to your project repository. One link per group. The grader will review the activity and history provided by GitHub. To add a hyperlink to a Jupyter markdown cell, follow the instructions in the cell below.
>  3. Below the GitHub Repository Link cell is a code cell. Copy and paste your final program code into this cell.

#### GitHub Repository Link
> Example: [INST326_Fall2024/Projects/Project03](https://github.com/sdempwolf/INST326_Fall_2024/tree/main/Projects/Project03)
>
> Edit the link code below with your information, then run this cell. Test the link! It should take you to your GitHub project repository.
> [external link text](http://url_here)

In [ ]:
# Solution - enter your code solution below


### Notebook Instructions
> Before turning in your notebook:
> 1. Make sure you have renamed the notebook file as instructed
> 2. Make sure you have included your signature block and that it is correct according to the instructions
> 3. comment your code as necessary
> 4. run all code cells and double check that they run correctly. If you can't get your code to run correctly and you want partial credit, add a note for the grader in a new markdown cell directly above your code solution.<br><br>
Turn in your notebook by uploading it to ELMS<br>
IF the exercises involve saved data files, put your notebook and the data file(s) in a zip folder and upload the zip folder to ELMS